# Exemple de génération de trace GPX à partir des points de la trace RTK exportée en format GeoJSON depuis l'application SW MAPS

Dans cette expérience, nous créons une trace depuis les points trackpoints collectés depuis SWMaps pour le Pôle d'Activités avec GoPro et RTK séparés l'un de l'autre pour vérifier l'impact sur les interférences et la qualité du signal RTK.

## Données d'entrée
Fichier geojson des points de la trace RTK collectés lors de la sortie de terrain - Pole d'activités d'Aix en Provence.

Fichier `..\qgis-projets\QGIS verification Precision Rover RTK et Photos GoPro\Données\Trace Points RTK et GoPro séparés 4326.geojson`

### 1) Installation de la bibliothèque python gpxpy pour la création de la trace GPX :

In [18]:
!pip install gpxpy

   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2) Installation de la bibliothèque python geojson pour le chargement du fichier geojson d'entrée :

In [20]:
!pip install geojson


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 3) Extraction des points RTK avec leurs horodatages en liste et les convertir en DataFrame :

In [4]:
import geojson
import pandas as pd

# Charger le fichier GeoJSON
with open(r"..\qgis-projets\QGIS verification Precision Rover RTK et Photos GoPro\Données\Trace Points RTK et GoPro séparés 4326.geojson", 'r') as f:
    data = geojson.load(f)

# Extraire les points et leurs horodatages
points = []
for feature in data['features']:
    if feature['geometry']['type'] == 'Point':
        coords = feature['geometry']['coordinates']
        timestamp = feature['properties'].get('time')
        points.append({
            'latitude': coords[1],
            'longitude': coords[0],
            'elevation': coords[2] if len(coords) > 2 else None,
            'time': timestamp
        })

# Convertir en DataFrame
df = pd.DataFrame(points)
df

,latitude,longitude,elevation,time
0,43.491060,5.377145,169.760,25-07-2024 12:25:14.800 GMT+02:00
1,43.491067,5.377112,169.729,25-07-2024 12:25:17.200 GMT+02:00
2,43.491076,5.377075,169.691,25-07-2024 12:25:19.600 GMT+02:00
3,43.491085,5.377040,169.606,25-07-2024 12:25:22.000 GMT+02:00
4,43.491093,5.377003,169.676,25-07-2024 12:25:24.400 GMT+02:00
...,...,...,...,...
71,43.491671,5.374420,167.742,25-07-2024 12:28:05.200 GMT+02:00
72,43.491680,5.374382,167.785,25-07-2024 12:28:07.600 GMT+02:00
73,43.491689,5.374343,167.932,25-07-2024 12:28:10.000 GMT+02:00
74,43.491699,5.374305,168.237,25-07-2024 12:28:12.400 GMT+02:00


### 4) Installation de la bibliothèque python dateutil pour le formatage de la date à partir d'une chaine de caractères :

In [40]:
pip install python-dateutil

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 5) Création de la trace GPX en ajoutant les points horodatées au segment de la trace :

In [5]:
import gpxpy
import gpxpy.gpx
from dateutil import parser

# Créer un objet GPX
gpx = gpxpy.gpx.GPX()

# Créer une nouvelle trace
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Ajouter un segment à la trace
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

# Ajouter les points au segment
for idx, row in df.iterrows():
# Convertir l'horodatage en format datetime
    time = parser.parse(row['time']) if row['time'] else None
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
        latitude=row['latitude'],
        longitude=row['longitude'],
        elevation=row['elevation'],
        time=time
    ))
print("end")

end


### 6) Exportation de la trace GPX :

In [6]:
# Écrire le fichier GPX
with open(r"..\qgis-projets\QGIS verification Precision Rover RTK et Photos GoPro\Données\Trace Points RTK et GoPro séparés.gpx", 'w') as f:
    f.write(gpx.to_xml())